In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets scipy
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [5]:
# loading the dataset

data = load_dataset("code_x_glue_tc_nl_code_search_adv", split="validation")
data_df = data.to_pandas()
data_df["text"] = data_df[["docstring", "code"]].apply(lambda x: "[INST] Docstring: " + x["docstring"] + " [/INST] Code: " + x["code"] + "", axis=1)
data = Dataset.from_pandas(data_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9604 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19210 [00:00<?, ? examples/s]

In [6]:
data

Dataset({
    features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'docstring_summary', 'parameters', 'return_statement', 'argument_list', 'identifier', 'nwo', 'score', 'text'],
    num_rows: 9604
})

In [7]:
data_df.head()

,id,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,docstring_summary,parameters,return_statement,argument_list,identifier,nwo,score,text
0,0,,src/you_get/extractors/dailymotion.py,,,python,"def Func(arg_0, arg_1='.', arg_2=True, arg_3=F...","[def, Func, (, arg_0, ,, arg_1, =, '.', ,, arg...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Downloads Dailymotion videos by URL.,"(url, output_dir='.', merge=True, info_only=Fa...",,,dailymotion_download,soimort/you-get,0.99976,[INST] Docstring: Downloads Dailymotion videos...
1,1,,src/you_get/extractors/ixigua.py,,,python,"def Func(arg_0):\n """"""Splicing URLs accordi...","[def, Func, (, arg_0, ), :, arg_1, =, [, """", ]...",Splicing URLs according to video ID to get vid...,"[Splicing, URLs, according, to, video, ID, to,...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Splicing URLs according to video ID to get vid...,(video_id),,,get_video_url_from_video_id,soimort/you-get,0.99976,[INST] Docstring: Splicing URLs according to v...
2,2,,src/you_get/common.py,,,python,"def Func(arg_0, arg_1={}, arg_2=True):\n """"...","[def, Func, (, arg_0, ,, arg_1, =, {, }, ,, ar...",Gets the content of a URL via sending a HTTP G...,"[Gets, the, content, of, a, URL, via, sending,...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,Gets the content of a URL via sending a HTTP G...,"(url, headers={}, decoded=True)",return data,,get_content,soimort/you-get,0.99976,[INST] Docstring: Gets the content of a URL vi...
3,3,,airflow/contrib/hooks/grpc_hook.py,,,python,"def Func(arg_0, arg_1, arg_2=None):\n ""...","[def, Func, (, arg_0, ,, arg_1, ,, arg_2, =, N...","Fetches a field from extras, and returns it. T...","[Fetches, a, field, from, extras, and, returns...",b69c686ad8a0c89b9136bb4b31767257eb7b2597,https://github.com/apache/airflow/blob/b69c686...,"Fetches a field from extras, and returns it. T...","(self, field_name, default=None)",,,GrpcHook._get_field,apache/airflow,0.99944,"[INST] Docstring: Fetches a field from extras,..."
4,4,,airflow/hooks/postgres_hook.py,,,python,"def Func(arg_0, arg_1, arg_2):\n """"""\n ...","[def, Func, (, arg_0, ,, arg_1, ,, arg_2, ), :...",Dumps a database table into a tab-delimited file,"[Dumps, a, database, table, into, a, tab, -, d...",b69c686ad8a0c89b9136bb4b31767257eb7b2597,https://github.com/apache/airflow/blob/b69c686...,Dumps a database table into a tab-delimited file,"(self, table, tmp_file)",,,PostgresHook.bulk_dump,apache/airflow,0.99944,[INST] Docstring: Dumps a database table into ...


In [8]:
data_df['language'].value_counts()

language
python    9604
Name: count, dtype: int64

In [11]:
# loading the codellama tokenizer and base model

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-hf", quantization_config=bnb_config, device_map={"":0})
model.config.use_cache=False
model.config.pretraining_tp=1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 252.00 MiB. GPU 

In [10]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )
training_arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/CodeBot/CodeLLama/codellama2-finetuned-codex",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=1,
    max_steps=100,
    fp16=True,
    push_to_hub=True
)
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512
)
trainer.train()

Map:   0%|          | 0/9604 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


OutOfMemoryError: CUDA out of memory. Tried to allocate 80.00 MiB. GPU 